In [ ]:
import xarray as xr
import numpy as np

import pyvista as pv
pv.set_jupyter_backend('client')

import cedalion
import cedalion.io
import cedalion.dataclasses as cdc
import cedalion.geometry.registration
import cedalion.geometry.segmentation
import cedalion.plots
import cedalion.xrutils as xrutils
import cedalion.imagereco.tissue_properties
import cedalion.imagereco.forward_model as fw
from cedalion import units
import pickle
import matplotlib.pyplot as p
import time

xr.set_options(display_expand_data=False);

## Load a finger-tapping dataset 
- Nasion landmark is renamed to match the landmark labels of the MRI scan
- give the optode positions a more descriptive CRS name

In [ ]:
elements = cedalion.io.read_snirf("../../data/BIDS-NIRS-Tapping/sub-01/nirs/sub-01_task-tapping_nirs.snirf")
geo3d_meas = elements[0].geo3d
geo3d_meas = geo3d_meas.points.rename({"NASION" : "Nz"})
geo3d_meas = geo3d_meas.rename({"pos" : "digitized"})
display(geo3d_meas)

The measurment list

In [ ]:
meas_list = elements[0].measurement_lists[0]
display(meas_list.head(5))

Select 20 seconds after a trial starts at t=117s and transform to optical density

In [ ]:
od = -np.log(elements[0].data[0] / elements[0].data[0].mean("time"))
od = od.sel(time=(od.time > 117) & (od.time < 137))
od

## Load MRI scan

- `cedalion.imagereco.forward_model.TwoSurfaceHeadModel` represents a segmented head with derived brain and scalp surfaces
- use TwoSurfaceHeadModel.from_segmentation to construct it
- mask_files map segmentation type label to niftii file
- the segmented volume is in unit-less voxel space ('ijk')
- it comes with an affine transformation t_ijk2ras which maps voxel space to RAS (right-anterior-superior) space with physical units.
- landmark position are expected to be in RAS space (that's the case when picked in slicer)

In [ ]:
#SEG_DATADIR = "../../AtlasViewerPy/demo_data/anatomy_data"
SEG_DATADIR = "../../data/mri/colin27_segmentation_nils"
head = fw.TwoSurfaceHeadModel.from_segmentation(
    segmentation_dir=SEG_DATADIR,
    mask_files = {
            "csf": "mask_csf.nii",
            "gm": "mask_gray.nii",
            "scalp": "mask_skin.nii",
            "skull": "mask_bone.nii",
            "wm": "mask_white.nii",
    },
    landmarks_ras_file="landmarks.mrk.json"
)

head.segmentation_masks

Landmark positions have been transformed to voxel space (ijk)

In [ ]:
head.landmarks

In [ ]:
head.crs

Register the opode positions to the scalp

In [ ]:
geo3d_snapped_ijk = head.align_and_snap_to_scalp(geo3d_meas)

In [ ]:

time.sleep(1)

plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_surface(plt, head.scalp, opacity=.1)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show()

## Simulate light propagation in tissue with MCX

`cedalion.imagereco.forward_model.ForwardModel` is a wrapper around pmcx. Using the data in the head model it preparse the inputs for pmcx and offers functionality to calculate the sensitivty matrix.

In [ ]:
fwm = cedalion.imagereco.forward_model.ForwardModel(head, geo3d_snapped_ijk, meas_list)

### Run the simulation

In [ ]:
fluence_all, fluence_at_optodes = fwm.compute_fluence()

simulated 100000000 photons (100000000) with 335872 threads (repeat x1)
MCX simulation speed: 98619.33 photon/ms
total simulated energy: 100000000.00	absorbed: 22.08482%
(loss due to initial specular reflection is excluded in the total)
simulating fluence for S5. 5 / 24
nphoton: 1e+08
tstart: 0
tstep: 5e-09
tend: 5e-09
isnormalized: 1
issrcfrom0: 1
unitinmm: 1
issavedet: 1
allocate 24579450 floats [12289725 1 0]
###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
########################

simulated 100000000 photons (100000000) with 335872 threads (repeat x1)
MCX simulation speed: 39277.30 photon/ms
total simulated energy: 100000000.00	absorbed: 47.93629%
(loss due to initial specular reflection is excluded in the total)
simulating fluence for D1. 9 / 24
nphoton: 1e+08
tstart: 0
tstep: 5e-09
tend: 5e-09
isnormalized: 1
issrcfrom0: 1
unitinmm: 1
issavedet: 1
allocate 24579450 floats [12289725 1 0]
###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
########################

simulating fluence for D5. 13 / 24
nphoton: 1e+08
tstart: 0
tstep: 5e-09
tend: 5e-09
isnormalized: 1
issrcfrom0: 1
unitinmm: 1
issavedet: 1
allocate 24579450 floats [12289725 1 0]
###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
###############################################################################
#    The MCX Project is funded by the NIH/NIGMS under grant R01-GM114365      #
###############################################################################
#  Open-source codes

tstart: 0
tstep: 5e-09
tend: 5e-09
isnormalized: 1
issrcfrom0: 1
unitinmm: 1
issavedet: 1
allocate 24579450 floats [12289725 1 0]
###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
###############################################################################
#    The MCX Project is funded by the NIH/NIGMS under grant R01-GM114365      #
###############################################################################
#  Open-source codes and reusable scientific data are essential for re

simulated 100000000 photons (100000000) with 335872 threads (repeat x1)
MCX simulation speed: 54229.93 photon/ms
total simulated energy: 100000000.00	absorbed: 33.79600%
(loss due to initial specular reflection is excluded in the total)
simulating fluence for D13. 21 / 24
nphoton: 1e+08
tstart: 0
tstep: 5e-09
tend: 5e-09
isnormalized: 1
issrcfrom0: 1
unitinmm: 1
issavedet: 1
allocate 24579450 floats [12289725 1 0]
###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2023 Qianqian Fang <q.fang at neu.edu>          #
#                https://mcx.space/  &  https://neurojson.org/                #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
######################

simulated 100000000 photons (100000000) with 335872 threads (repeat x1)
MCX simulation speed: 36456.43 photon/ms
total simulated energy: 100000000.00	absorbed: 47.92744%
(loss due to initial specular reflection is excluded in the total)


## Plot fluence

In [ ]:
time.sleep(1)

plt = pv.Plotter()

f = fluence_all[0,0,:,:,:].values * fluence_all[8,0,:,:,:].values
f[f==0] = f[f!=0].min()
f = np.log10(f)
vf = pv.wrap(f)

plt.add_volume(
    vf,
    log_scale=False, 
    cmap='plasma_r', # 'gist_earth_r', 
    clim=(-10,0),
)
cedalion.plots.plot_surface(plt, head.brain, color="w")
#cedalion.plots.plot_surface(plt, head.scalp, opacity=.1)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show()

### Calculate the sensitivity matrix

In [ ]:
Adot = fwm.compute_sensitivity(fluence_all, fluence_at_optodes)
Adot

The sensitivity matrix `Adot`has shape (nchannel, nvertex, nwavelenghts). It does not yet include the extinction coefficients. Stack the channel and wavelength dimensions as well as the vertex and chromophore dimensions into new dimensions (flat_channel, flat_vertex)

In [ ]:
Adot_stacked = fwm.compute_stacked_sensitivity(Adot)
Adot_stacked

### Invert the sensitivity matrix

In [ ]:
def pseudo_inverse_stacked(Adot, alpha=0.01):
    AA = Adot.values @ Adot.values.T
    highest_eigenvalue = np.linalg.eig(AA)[0][0].real
    
    B = (Adot.values.T @ np.linalg.pinv(AA + alpha*highest_eigenvalue*np.eye(AA.shape[0])))
    B = xr.DataArray(B, dims=("flat_vertex", "flat_channel"))
    
    return B

B = pseudo_inverse_stacked(Adot_stacked)
B

### Calculate concentration changes

- the optical density has shape (nchannel, nwavelength, time) -> stack it

In [ ]:
od_stacked = od.stack({"flat_channel" : ["wavelength", "channel"]})
dC = B @ od.stack({"flat_channel" : ["wavelength", "channel"]})
dC

## Plot concentration changes

In [ ]:
b = cdc.VTKSurface.from_trimeshsurface(head.brain)
b = pv.wrap(b.mesh)
s = cdc.VTKSurface.from_trimeshsurface(head.scalp)
s = pv.wrap(s.mesh)

hbo = dC[:dC.shape[0]//2, :]
hbo_brain = hbo[(Adot.is_brain == True).values,:]
hbo_scalp = hbo[(Adot.is_brain == False).values,:]

plt = pv.Plotter()

i = 10
tt = od.time[i]
b["reco_hbo"] = hbo_brain[:,i] - hbo_brain[:,0]
s["reco_hbo"] = hbo_scalp[:,i] - hbo_scalp[:,0]
    
# plot brain surface
plt = pv.Plotter()
    
plt.add_mesh(
    b,
    scalars="reco_hbo",
    cmap='seismic', # 'gist_earth_r', 
    clim=(-10e-7,10e-7),
)

cog = head.brain.vertices.mean("label").values
plt.camera.position = cog + [0,0,400]
plt.camera.focal_point = cog 
plt.camera.up = [0,1,0] 
plt.reset_camera()

plt.add_text(f"time: {tt:.3f} s")
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show()

plt = pv.Plotter()

i = 10
tt = od.time[i]
b["reco_hbo"] = hbo_brain[:,i] - hbo_brain[:,0]
s["reco_hbo"] = hbo_scalp[:,i] - hbo_scalp[:,0]
    

# plot scalp surface
plt = pv.Plotter()
    
plt.add_mesh(
    s,
    scalars="reco_hbo",
    cmap='seismic', # 'gist_earth_r', 
    clim=(-3e-6,3e-6),
)

cog = head.brain.vertices.mean("label").values
plt.camera.position = cog + [0,0,400]
plt.camera.focal_point = cog 
plt.camera.up = [0,1,0] 
plt.reset_camera()

plt.add_text(f"time: {tt:.3f} s")
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show()